In [2]:
import torch 
import torchvision 
import torch.nn as nn 
import torch.optim as optim 
import torch.nn.functional as F

In [3]:
train_dataset = torchvision.datasets.MNIST(root='./MNIST/train',  
                                           train=True,  
                                           transform=torchvision.transforms.ToTensor(),  
                                           download=False) 
test_dataset = torchvision.datasets.MNIST(root='./MNIST/test',  
                                          train=False,  
                                          transform=torchvision.transforms.ToTensor(),  
                                          download=False)

print("Datasets are upToDate")

Datasets are upToDate


In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
device

device(type='cuda', index=0)

In [18]:
class ConvClassif(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        self.fc = nn.Linear(32*28*28, 10)

    def forward(self,x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = torch.flatten(x,1)
        x = self.fc(x)
        return F.softmax(x, dim=1)

model = ConvClassif().to(device)

criterion = nn.CrossEntropyLoss() 
optimizer = optim.Adam(model.parameters(), lr=0.001)

batch_size = 1000
num_epochs = 30

val_percent = 0.2
val_size    = int(val_percent * len(train_dataset)) 
train_size  = len(train_dataset) - val_size 
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset,  
                                                           [train_size,  
                                                            val_size])

train_loader = torch.utils.data.DataLoader(train_dataset,  
                                           batch_size=batch_size,  
                                           shuffle=True, 
                                           pin_memory=True) 
val_loader = torch.utils.data.DataLoader(val_dataset,  
                                         batch_size=batch_size,  
                                         shuffle=False, 
                                         pin_memory=True)

losses = [] 
accuracies = [] 
val_losses = [] 
val_accuracies = []

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = correct / total
    losses.append(epoch_loss)
    accuracies.append(epoch_acc)

    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            val_correct += (predicted == labels).sum().item()
            val_total += labels.size(0)

    val_epoch_loss = val_loss / len(val_loader)
    val_epoch_acc = val_correct / val_total
    val_losses.append(val_epoch_loss)
    val_accuracies.append(val_epoch_acc)
      
              
    print('Epoch [{}/{}],Loss:{:.4f},Validation Loss:{:.4f},Accuracy:{:.2f},Validation Accuracy:{:.2f}'.format( 
        epoch+1, num_epochs, epoch_loss, val_epoch_loss, epoch_acc ,val_epoch_acc))
    

        
        

Epoch [1/30],Loss:1.8546,Validation Loss:1.7150,Accuracy:0.65,Validation Accuracy:0.75
Epoch [2/30],Loss:1.7067,Validation Loss:1.6936,Accuracy:0.75,Validation Accuracy:0.77
Epoch [3/30],Loss:1.6969,Validation Loss:1.6872,Accuracy:0.76,Validation Accuracy:0.77
Epoch [4/30],Loss:1.6913,Validation Loss:1.6838,Accuracy:0.77,Validation Accuracy:0.78
Epoch [5/30],Loss:1.6864,Validation Loss:1.6782,Accuracy:0.78,Validation Accuracy:0.78
Epoch [6/30],Loss:1.6797,Validation Loss:1.6735,Accuracy:0.78,Validation Accuracy:0.79
Epoch [7/30],Loss:1.6755,Validation Loss:1.6702,Accuracy:0.79,Validation Accuracy:0.79
Epoch [8/30],Loss:1.5774,Validation Loss:1.5280,Accuracy:0.89,Validation Accuracy:0.94
Epoch [9/30],Loss:1.5119,Validation Loss:1.5110,Accuracy:0.95,Validation Accuracy:0.95
Epoch [10/30],Loss:1.4994,Validation Loss:1.5016,Accuracy:0.97,Validation Accuracy:0.96
Epoch [11/30],Loss:1.4927,Validation Loss:1.4997,Accuracy:0.97,Validation Accuracy:0.96
Epoch [12/30],Loss:1.4890,Validation Loss

In [20]:
test_loader = torch.utils.data.DataLoader(test_dataset,  
                                          batch_size=batch_size,  
                                          shuffle=False) 
  
model.eval() 
  
with torch.no_grad(): 
    correct = 0
    total = 0
    y_true = [] 
    y_pred = [] 
    for images, labels in test_loader: 
        images = images.to(device) 
        labels = labels.to(device) 
        outputs = model(images) 
        _, predicted = torch.max(outputs.data, 1) 
        total += labels.size(0) 
        correct += (predicted == labels).sum().item() 
        predicted=predicted.to('cpu') 
        labels=labels.to('cpu') 
        y_true.extend(labels) 
        y_pred.extend(predicted) 
  
print('Test Accuracy: {}%'.format(100 * correct / total)) 
  
from sklearn.metrics import classification_report 
print(classification_report(y_true, y_pred))

Test Accuracy: 98.01%
              precision    recall  f1-score   support

           0       0.97      0.99      0.98       980
           1       0.98      0.99      0.99      1135
           2       0.97      0.97      0.97      1032
           3       0.97      0.98      0.98      1010
           4       0.98      0.99      0.99       982
           5       0.99      0.97      0.98       892
           6       0.98      0.98      0.98       958
           7       0.98      0.97      0.97      1028
           8       0.98      0.97      0.98       974
           9       0.98      0.97      0.98      1009

    accuracy                           0.98     10000
   macro avg       0.98      0.98      0.98     10000
weighted avg       0.98      0.98      0.98     10000

